This notebook can be run on Google Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nanotheorygroup/kaldo/blob/master/docs/docsource/crystal_presentation.ipynb)



In Colab, you can enable the GPU acceleration from `Edit` > `Notebook Settings` > `Accelerator` > `GPU`.

# Silicon diamond tutorial

## Remote fetch source code from Github

In [ ]:
!pip install git+https://github.com/nanotheorygroup/kaldo

## Remote fetch supplyment files


In [ ]:
import os
if not os.path.exists('forcefields.zip'):
  !wget http://sophe.ucdavis.edu/structures/forcefields.zip
  !unzip forcefields.zip
  !rm -r forcefields.zip

## Remote fetch precomplied LAMMPS


In [ ]:
!wget http://sophe.ucdavis.edu/structures/lmp-tesla-t4-intel-xeon.gz
!tar xvzf lmp-tesla-t4-intel-xeon.gz
!rm /content/lmp-tesla-t4-intel-xeon.gz
!ln -s /content/lammps/src/lmp_gpu /usr/local/bin/lmp_gpu

# Navigate back to lammps source foder when for 
# later LAMMPS-Python integration

%cd /content/lammps/src

print('\n')
print('Remote Fetching precomplied LAMMPS is finished!')

## Integrate LAMMPS Into Python 

In [ ]:
!make install-python

# Copy executable file to where the python Module locates

import shutil 
src_path = '/usr/lib/python3/dist-packages/liblammps.so'
dist_path = '/usr/local/lib/python3.7/dist-packages/liblammps.so'
shutil.copyfile(src_path,dist_path)

# Naviagate back to main folder before simulation

%cd /content

print('\n')
print('LAMMPS-Python Integration is completed!')

## Thermal transport simulation for silicon-bulk

### Compute $2^{nd}$ and $3^{rd}$ order IFCS.

In [ ]:
from ase.build import bulk
from ase.calculators.lammpslib import LAMMPSlib
from kaldo.forceconstants import ForceConstants
import numpy as np

# We start from the atoms object

atoms = bulk('Si', 'diamond', a=5.432)

# Config super cell and calculator input

supercell = np.array([3, 3, 3])
lammps_inputs = {
      'lmpcmds': [
          'pair_style tersoff',
          'pair_coeff * * forcefields/Si.tersoff Si'],

      'log_file': 'lammps-si-bulk.txt',
      'keep_alive':True}

# Create a finite difference object

forceconstants_config  = {'atoms':atoms,'supercell': supercell,'folder':'fd'}
forceconstants = ForceConstants(**forceconstants_config)

# Compute 2nd and 3rd IFCs with the defined calculators

forceconstants.second.calculate(LAMMPSlib(**lammps_inputs), delta_shift=1e-4)
forceconstants.third.calculate(LAMMPSlib(**lammps_inputs), delta_shift=1e-4)

### Create phonons object


In [7]:
from kaldo.phonons import Phonons

# Define k-point grids, temperature
# and the assumption for the 
# phonon poluation (i.e classical vs. quantum)

k = 7
kpts = [k, k, k]
temperature = 300
is_classic = False
k_label = str(k) + '_' + str(k) + '_' + str(k)

# Create a phonon object

phonons = Phonons(forceconstants=forceconstants,
                kpts=kpts,
                is_classic=is_classic,
                temperature=temperature,
                folder='si-bulk-ald-' + k_label,
                storage='numpy')

### Calculate conductivities for infinite-size sample


In [ ]:
from kaldo.conductivity import Conductivity

# Calculate conductivity  with direct inversion approach (inverse)

print('\n')
inv_cond_matrix = (Conductivity(phonons=phonons, method='inverse').conductivity.sum(axis=0))
print('Inverse conductivity (W/mK): %.3f'%(np.mean(np.diag(inv_cond_matrix))))
print(inv_cond_matrix)

# Calculate conductivity  with  relaxation time approximation (rta)

print('\n')
rta_cond_matrix = Conductivity(phonons=phonons, method='rta').conductivity.sum(axis=0)
print('Rta conductivity (W/mK): %.3f'%(np.mean(np.diag(rta_cond_matrix))))
print(rta_cond_matrix)

# Calculate conductivity  with  self-consistent approach (sc)

print('\n')
sc_cond_matrix = Conductivity(phonons=phonons, method='sc',n_iterations=20).conductivity.sum(axis=0)
print('Self-consistent conductivity (W/mK): %.3f'%(np.mean(np.diag(sc_cond_matrix))))
print(sc_cond_matrix)

### Visualize harmonic properties using built-in plotter


In [ ]:
import kaldo.controllers.plotter as plotter
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')

# Plot dispersion relation and group velocity in each direction

plotter.plot_dispersion(phonons,n_k_points=int(k_label))
print('\n')

### Access and visualize properties calculated during simulations

In [ ]:
# Direct access to properties
# calculated during the simulation

# Plot heat capacity vs frequency

freq_full = phonons.frequency.flatten(order='C')
cv_1d = phonons.heat_capacity.flatten(order='C')[3:]

print('\n')
plt.figure()
plt.scatter(freq_full[3:],1e23*cv_1d,s=15)
plt.ylabel (r"$C_{v}$ ($10^{23}$ J/K)", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylim(0.9*1e23*cv_1d[cv_1d>0].min(), 
         1.05*1e23*cv_1d.max())
plt.show()

# Plot phonon bandwidth vs frequency

band_wdidth_flatten = phonons.bandwidth.flatten(order='C')
freq = freq_full[band_wdidth_flatten!=0]

print('\n')
plt.figure()
plt.scatter(freq,band_wdidth_flatten[band_wdidth_flatten!=0] ,s=15)
plt.ylabel (r"$\Gamma$ (Thz)", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylim(0.95*band_wdidth_flatten .min(), 1.05*band_wdidth_flatten .max())
plt.show()

# Plot phase space vs frequency

print('\n')
plt.figure()
plt.scatter(freq_full[3:],phonons.phase_space.flatten(order='C')[3:],s=15)
plt.ylabel ("Phase space", fontsize=25, fontweight='bold')
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylim(phonons.phase_space.min(), phonons.phase_space.max())
plt.show()

### Calculate and visualize $\kappa_{per \ mode}$ and $\kappa_{cum}$

In [11]:
def cumulative_cond_cal(freq,full_cond,n_phonons):

  conductivity = np.einsum('maa->m', 1/3 * full_cond)
  conductivity = conductivity.reshape(n_phonons)
  cumulative_cond = np.zeros_like(conductivity)
  freq_reshaped = freq.reshape(n_phonons)

  for mu in range(cumulative_cond.size):
      single_cumulative_cond = conductivity[(freq_reshaped < freq_reshaped[mu])].sum()
      cumulative_cond[mu] = single_cumulative_cond
  
  return cumulative_cond

In [ ]:
# Compute conductivity with per phonon mode basis using different methods

kappa_rta_per_mode = np.einsum('maa->m',1/3*Conductivity(phonons=phonons, method='rta').conductivity)
kappa_inv_per_mode = np.einsum('maa->m',1/3*Conductivity(phonons=phonons, method='inverse').conductivity)
kappa_sc_per_mode = np.einsum('maa->m',1/3*Conductivity(phonons=phonons, method='sc',n_iterations=20).conductivity)

# Compute cumulative conductivity by frequency using different methods

kappa_rta_cum_freq = cumulative_cond_cal(phonons.frequency,Conductivity(phonons=phonons, method='rta').conductivity,phonons.n_phonons)
kappa_sc_cum_freq = cumulative_cond_cal(phonons.frequency,Conductivity(phonons=phonons, method='sc',n_iterations=20).conductivity,phonons.n_phonons)
kappa_inv_cum_freq = cumulative_cond_cal(phonons.frequency,Conductivity(phonons=phonons, method='inverse').conductivity,phonons.n_phonons)
kappa_qhgk_cum_freq = cumulative_cond_cal(phonons.frequency,Conductivity(phonons=phonons, method='qhgk').conductivity,phonons.n_phonons)

print('\n')

# Visualize the cumulative conductivity vs frequency

plt.figure()
plt.plot(freq_full,kappa_rta_per_mode,'r.',label='RTA')
plt.plot(freq_full,kappa_sc_per_mode,'mo',label='Self Consistent',ms=8)
plt.plot(freq_full,kappa_inv_per_mode,'k.',label='Direct Inversion')
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylabel(r'$\kappa(W/m/K)$', fontsize=25, fontweight='bold')
plt.legend(loc=1,frameon=False)
#plt.grid()
plt.show()

print('\n')

# Visualize the cumulative conductivity vs frequency

plt.figure()
plt.plot(freq_full,kappa_rta_cum_freq,'r.',label='RTA')
plt.plot(freq_full,kappa_sc_cum_freq,'mo',label='Self Consistent',ms=8)
plt.plot(freq_full,kappa_inv_cum_freq,'k.',label='Direct Inversion')
plt.xlabel ("$\\nu$ (Thz)", fontsize=25, fontweight='bold')
plt.ylabel(r'$\kappa_{cum}(W/m/K)$', fontsize=25, fontweight='bold')
plt.legend(loc=4,frameon=False)
plt.grid()
plt.show()